In [1]:
import torch
import numpy as np

In [2]:
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

targets = np.array([[56, 70],
                   [81, 101],
                   [119, 133],
                   [22, 37],
                   [103, 119]], dtype='float32')

In [3]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [6]:
# 2 rows, 3 columns
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

In [7]:
def model(x):
    return x @ w.t() + b

In [9]:
preds = model(inputs)
print(preds)
print(targets)

tensor([[ -80.2063,    5.5263],
        [-107.4668,    0.8405],
        [-134.3834,   -9.9305],
        [ -68.7164,   31.2204],
        [-110.0475,  -16.8356]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [10]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [13]:
loss = mse(targets, preds)
print(loss)

tensor(22499.6367, grad_fn=<DivBackward0>)


In [14]:
loss.backward()

In [16]:
print(w)
print(w.grad)

tensor([[-0.2407, -0.6621, -0.4297],
        [ 0.5838, -0.2457, -0.4778]], requires_grad=True)
tensor([[-14618.2480, -16803.5293, -10176.9629],
        [ -7243.6445,  -9115.0391,  -5438.9761]])


In [17]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [18]:
# Training

preds = model(inputs)
print(preds)

loss = mse(preds, targets)
print(loss)

tensor([[ -80.2063,    5.5263],
        [-107.4668,    0.8405],
        [-134.3834,   -9.9305],
        [ -68.7164,   31.2204],
        [-110.0475,  -16.8356]], grad_fn=<AddBackward0>)
tensor(22499.6367, grad_fn=<DivBackward0>)


In [19]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-14618.2480, -16803.5293, -10176.9629],
        [ -7243.6445,  -9115.0391,  -5438.9761]])
tensor([-176.3641,  -89.8358])


In [20]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [21]:
print(w)
print(b)

tensor([[-0.0945, -0.4941, -0.3279],
        [ 0.6562, -0.1546, -0.4234]], requires_grad=True)
tensor([ 0.2041, -0.0812], requires_grad=True)


In [22]:
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [23]:
preds = model(inputs)
loss = mse(preds, targets)
print(preds)
print(loss)

tensor([[ 62.5274,  77.1223],
        [ 82.8598,  98.6215],
        [108.5882, 126.6641],
        [ 51.7449,  76.0228],
        [ 85.2035,  92.7948]], grad_fn=<AddBackward0>)
tensor(366.1965, grad_fn=<DivBackward0>)


In [73]:
for i in range(1000000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    
    with torch.no_grad():
        w -= w.grad * 1e-7
        b -= b.grad * 1e-7
        w.grad.zero_()
        b.grad.zero_()

In [74]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(0.5172, grad_fn=<DivBackward0>)


In [75]:
import torch.nn as nn

In [77]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [79]:
from torch.utils.data import TensorDataset

In [82]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [83]:
train_ds[[1,3,5,7]]

(tensor([[ 91.,  88.,  64.],
         [102.,  43.,  37.],
         [ 73.,  67.,  43.],
         [ 87., 134.,  58.]]),
 tensor([[ 81., 101.],
         [ 22.,  37.],
         [ 56.,  70.],
         [119., 133.]]))

In [84]:
from torch.utils.data import DataLoader

In [85]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [86]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  88.,  64.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 73.,  67.,  43.],
        [ 69.,  96.,  70.]])
tensor([[ 81., 101.],
        [ 22.,  37.],
        [ 56.,  70.],
        [ 56.,  70.],
        [103., 119.]])


In [87]:
# Model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2646, -0.5443,  0.3950],
        [ 0.3078,  0.5347,  0.3806]], requires_grad=True)
Parameter containing:
tensor([ 0.2821, -0.0482], requires_grad=True)


In [89]:
preds = model(inputs)
preds

tensor([[-38.5192,  74.6158],
        [-46.4179,  99.3781],
        [-72.7694, 120.4612],
        [-35.4986,  68.4263],
        [-42.5812,  99.1667],
        [-38.5192,  74.6158],
        [-46.4179,  99.3781],
        [-72.7694, 120.4612],
        [-35.4986,  68.4263],
        [-42.5812,  99.1667],
        [-38.5192,  74.6158],
        [-46.4179,  99.3781],
        [-72.7694, 120.4612],
        [-35.4986,  68.4263],
        [-42.5812,  99.1667]], grad_fn=<AddmmBackward>)

In [90]:
import torch.nn.functional as F

In [91]:
loss_fn = F.mse_loss

In [92]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(8800.6826, grad_fn=<MseLossBackward>)


In [93]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [95]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2646, -0.5443,  0.3950],
         [ 0.3078,  0.5347,  0.3806]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2821, -0.0482], requires_grad=True)]

In [96]:
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            pred = model(xb)
            
            loss = loss_fn(pred, yb)
            
            loss.backward()
            
            opt.step()
            
            opt.zero_grad()
            
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [98]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 16.3587
Epoch [20/100], Loss: 18.5526
Epoch [30/100], Loss: 11.5948
Epoch [40/100], Loss: 6.9515
Epoch [50/100], Loss: 7.6111
Epoch [60/100], Loss: 7.7979
Epoch [70/100], Loss: 5.0681
Epoch [80/100], Loss: 6.2525
Epoch [90/100], Loss: 2.4196
Epoch [100/100], Loss: 4.9079


In [99]:
preds = model(inputs)

In [100]:
preds

tensor([[ 57.2638,  70.6929],
        [ 83.0521,  98.7986],
        [116.4438, 136.8307],
        [ 21.5929,  38.8965],
        [103.1603, 114.7571],
        [ 57.2638,  70.6929],
        [ 83.0521,  98.7986],
        [116.4438, 136.8307],
        [ 21.5929,  38.8965],
        [103.1603, 114.7571],
        [ 57.2638,  70.6929],
        [ 83.0521,  98.7986],
        [116.4438, 136.8307],
        [ 21.5929,  38.8965],
        [103.1603, 114.7571]], grad_fn=<AddmmBackward>)

In [101]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])